# 1.1 Amazon SageMaker Training 실습

## 학습 작업의 실행 노트북 개요

이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)

- 일반적으로 SageMaker Training은 아래와 같이 진행이 됩니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)

### 작업 실행 시 필요 라이브러리 import

In [4]:
import boto3
import sagemaker
print(f"SageMaker Version: {sagemaker.__version__}")

SageMaker Version: 2.227.0


### SageMaker 세션과 Role, 사용 버킷 정의

In [2]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
print(f'role: {role}')

role: arn:aws:iam::242201274000:role/service-role/AmazonSageMaker-ExecutionRole-20240923T230631


In [5]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code' # 코드 백업 저장소
output_path = f's3://{bucket}/xgboost/output' # 모델, 아웃풋 (tar.gz 파일 저장 위치)

print(f"default bucket: {bucket}")
print(f"code location: {code_location}")
print(f"output path: {output_path}")

default bucket: sagemaker-ap-northeast-2-242201274000
code location: s3://sagemaker-ap-northeast-2-242201274000/xgboost/code
output path: s3://sagemaker-ap-northeast-2-242201274000/xgboost/output


### 하이퍼파라미터 정의

In [7]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2", #learning rate
        "objective": "binary:logistic", # 알고리즘: 이진분류
        "num_round": "100", # iterations
}
# XGBoost 쓸 떄 필요한 하이퍼 파라미터
# src/xgboost_starter_script.py 에서 쓰는 args로 들어감!
hyperparameters

{'scale_pos_weight': '29',
 'max_depth': '3',
 'eta': '0.2',
 'objective': 'binary:logistic',
 'num_round': '100'}

### 학습 실행 작업 정의

In [9]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = "local"
max_run = 1*60*60 # 최대 학습 실행 시간: 최대 1시간

use_spot_instances = False # spot instance 쓸지 말지
if use_spot_instances:
    max_wait = 1*60*60 # spot 인스턴스 쓸때만 설정해주면 됨
else:
    max_wait = None

In [10]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

In [11]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",  # 1. entry_point: 학습 클러스터에서 사용할 python script 이름
    source_dir='src',                         # 2. source_dir: 노트북 인스턴스 상에서 entry_point가 저장되어 있는 위치
    output_path=output_path,                  # 3. output_path: 학습 클러스터에서 /opt/ml/model, /opt/ml/output에 저장한 내용 복사할 S3 위치
    code_location=code_location,              # 4. code_location: 학습에 사용한 스크립트를 백업 저장할 S3 위치
    hyperparameters=hyperparameters,          # 5. hyperparameters: 내 python script에 넣어줄 args
    role=role,                                # 6. role: 학습 클러스터가 사용할 수 있는 role
    sagemaker_session=sagemaker_session,      # 7. sagemaker_session: 세션 설정. LocalSession으로 하면  테스트 모드임
    instance_count=instance_count,            # 8. instance_count: 학습 클러스터를 구성하는 인스턴스 개수
    instance_type=instance_type,              # 9. instance_type: 학습 클러스터를 구성하는 인스턴스 타입
    framework_version="1.3-1",                # 10. framework_version: 도커 이미지 검색에 사용됨. 여기서는 XGBoost의 버전을 의미
    max_run=max_run,                          # 11. max_run: 최대 실행 시간
    use_spot_instances=use_spot_instances,    # 12. use_spot_instances: spot instance 활용여부
    max_wait=max_wait,                        # 13. max_wait: use_spot_instances를 true로 설정했을 때, 최대 대기 시간
)

### 학습 데이터셋 지정

In [12]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ../data/dataset/ $data_path

upload: ../data/dataset/test.csv to s3://sagemaker-ap-northeast-2-242201274000/xgboost/dataset/test.csv
upload: ../data/dataset/train.csv to s3://sagemaker-ap-northeast-2-242201274000/xgboost/dataset/train.csv


In [13]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
    
else:
    inputs = data_path
inputs

's3://sagemaker-ap-northeast-2-242201274000/xgboost/dataset'

### 학습 실행

In [14]:
# training 실행 --> inputs로 노트북 인스턴스의 관점에서 input 경로를 넣어줒ㅁ
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-11-06-01-33-30-891


In [15]:
estimator.logs()

2024-11-06 01:33:32 Starting - Starting the training job...
2024-11-06 01:34:10 Downloading - Downloading input data......
2024-11-06 01:34:50 Downloading - Downloading the training image...
2024-11-06 01:35:36 Training - Training image download completed. Training in progress..[2024-11-06 01:35:41.738 ip-10-0-246-31.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-11-06 01:35:41.765 ip-10-0-246-31.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-11-06:01:35:41:INFO] Imported framework sagemaker_xgboost_container.training
[2024-11-06:01:35:41:INFO] No GPUs detected (normal if no gpus installed)
[2024-11-06:01:35:41:INFO] Invoking user training script.
[2024-11-06:01:35:42:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2024-11-06:01:35:42:INFO] Generating setup.cfg
[2024-11-06:01:35:42:INFO] Generating MANIFEST.in
[2024-11-06:01:35:42:INFO] Installi

### 데이터 세트 설명
- 데이터 세트는 블로그 [Architect and build the full machine learning lifecycle with AWS: An end-to-end Amazon SageMaker demo](https://aws.amazon.com/blogs/machine-learning/architect-and-build-the-full-machine-learning-lifecycle-with-amazon-sagemaker/) 에서 사용한 데이터를 사용합니다. 블로그에서는 데이터 세트에 대해서 이렇게 설명 합니다.
- "자동차 보험 청구 사기를 탐지를 위해서 블로그 저자가 데이터를 합성해서 만든 고객과 클레임의 데이터 세트를 사용합니다."

In [16]:
import pandas as pd

In [17]:
train_prep_df = pd.read_csv('../data/dataset/train.csv')
train_prep_df.groupby('fraud').sample(n=5)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
461,0,16206.847582,35406.847582,36,52,0,1,750,2750,91744,...,0,1,0,0,0,0,0,1,1,0
1956,0,4905.687258,12505.687258,42,134,0,1,750,3000,89122,...,0,0,1,0,0,0,1,0,1,0
541,0,40758.992939,61458.992939,43,28,0,1,750,3000,91762,...,0,0,0,1,1,0,0,0,1,0
2295,0,29115.411283,64215.411283,48,108,0,1,750,3000,93611,...,0,0,0,1,0,1,0,0,1,0
2489,0,15337.661464,31137.661464,48,76,0,1,750,3000,97062,...,0,0,1,0,0,0,1,0,1,0
3930,1,8608.986192,38608.986192,39,91,0,1,750,3000,92592,...,0,0,1,0,0,0,1,0,1,0
2178,1,8550.257202,31550.257202,20,9,0,4,750,2750,92567,...,0,0,1,0,0,0,1,0,1,0
737,1,4716.911325,17716.911325,36,4,0,4,750,3000,98034,...,1,0,1,0,0,0,0,1,1,0
2235,1,8337.213616,29237.213616,26,2,0,2,750,3000,91423,...,0,0,1,0,0,0,1,0,1,0
1987,1,14359.535421,17759.535421,24,38,0,2,750,3000,94110,...,0,1,0,0,0,0,0,1,0,1


In [18]:
train_prep_df.groupby('fraud').size()

fraud
0    3869
1     131
dtype: int64

#### 데이터 컬럼 설명
- fraud: 보험 청구의 사기 여부 입니다. 1 이면 사기, 0 이면 정상 청구 입니다.
- vehicle_claim: 자동차에 대한 보험 청구액. 값으로서, $1000, $17,638 등이 있습니다.
- total_claim_amount: 전체 보험 청구액 입니다. $21,400, $10,000 등이 있습니다.    
- customer_age: 고객의 나이를 의미 합니다.
- months_as_customer: 고객으로서의 가입 기간을 의미합니다. 단위는 월로서 11, 30, 31 등의 값이 존재 합니다.
- num_claims_past_year: 작년의 보험 청구 수를 의미 합니다. 0, 1, 2, 3, 4, 5, 6 의 값이 존재 합니다.
- num_insurers_past_5_years: 과거 5년 동안의 보험 가입 회사 수를 의미 합니다. 1, 2, 3, 4, 5 의 값이 존재 합니다.
- policy_deductable: 보험의 최소 자기 부담금 입니다. $750, $800 등이 있습니다.    
- policy_annual_premium: 보험의 특약 가입에 대한 금액 입니다. $2000, $3000 등이 있습니다.
- customer_zip: 고객의 집 주소 우편 번호를 의미합니다.
- auto_year: 자동차의 년식을 의미 합니다. 2020, 2019 등이 있습니다.
- num_vehicles_involved: 몇 대의 자동차가 사고에 연관 되었는지 입니다. 1, 2, 3, 4, 5, 6 의 값이 있습니다.
- num_injuries: 몇 명이 상해를 입었는지를 기술합니다. 0, 1, 2, 3, 4, 의 값이 있습니다.
- num_witnesses: 몇 명의 목격자가 있었는지를 기술합니다. 0, 1, 2, 3, 4, 5 의 값이 있습니다.
- injury_claim: 상해에 대한 보험 청구액. \$5,500, \$70,700, \$100,700 등이 있습니다.    
- incident_month: 사고가 발생한 월을 의미합니다. 1~12 값이 존재 합니다.
- incident_day: 사고가 발생한 일자를 의미합니다. 1~31 값이 존재 합니다.
- incident_dow: 사고가 발생한 요일을 의미합니다. 0~6 값이 존재 합니다.
- incident_hour: 사고가 발생한 시간을 의미합니다. 0~23 값이 존재 합니다.
- policy_state: 보험 계약을 한 미국 주(State)를 의미 합니다. CA, WA, AZ, OR, NV, ID 가 존재 합니다.    
- policy_liability: 보험 청구의 한도를 의미 합니다. 예를 들어서 25/50 은  사람 당 상해 한도 $25,000, 사고 당 상해 한도가 $50,000 을  의미합니다. 25/50, 15/30, 30/60, 100/200 의 값이 존재 합니다. 
- customer_gender: 고객의 성별을 의미 합니다. Male, Female, Unkown, Other가 존재 합니다.
- customer_education: 고객의 최종 학력을 의미합니다. Bachelor, High School, Advanced Degree, Associate, Below High School 이 존재 합니다.
- driver_relationship: 보험 계약자와 운전자와의 관계 입니다. Self, Spouse, Child, Other 값이 존재 합니다.
- incident_type: 사고의 종류를 기술합니다. Collision, Break-in, Theft 값이 존재 합니다.
- collision_type: 충돌 타입을 기술합니다. Front, Rear, Side, missing 값이 존재 합니다.
- incident_severity: 사고의 손실 정도 입니다. Minor, Major, Totaled 값이 존재 합니다.
- authorities_contacted: 어떤 관련 기관에 연락을 했는지 입니다. Police, Ambuylance, Fire, None 값이 존재 합니다.
- police_report_available: 경찰 보고서가 존재하는지를 기술합니다. Yes, No 의 값이 있습니다.